In [20]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from scipy.stats import sem
import os
# import DataProcessor as dp

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

In [73]:
def retrieve_summary(zone_lkup_file, taxi_file_names, gb_field):
    taxi_zone_lkup_df = pd.read_csv(taxi_zone_lkup_csv)
    # chunk_list = []  # append each chunk df here     
    gb_field_list = ["trip_count","total_revenue","total_time_min","total_miles"]

    gb_final_file_df = pd.DataFrame()

    for file_key in taxi_file_names: 
        clean_file_df = clean_file(taxi_file_names[file_key], taxi_zone_lkup_df, file_key, gb_field)
        gb_clean_file_df = groupby(clean_file_df, gb_field, gb_field_list)
        gb_final_file_df = gb_final_file_df.append(gb_clean_file_df)
        print(file_key)

    gb_final_file_df = groupby(gb_final_file_df, gb_field, gb_field_list)
    gb_final_file_df = enrich_summary(gb_final_file_df)
    
    return gb_final_file_df

def enrich_summary(df):
    new_df = df
    new_df['avg_min_ptrip'] = new_df['total_time_min']/new_df['trip_count']
    new_df['avg_dist_ptrip'] = new_df['total_miles']/new_df['trip_count']
    new_df['avg_rev_ptrip'] = new_df['total_revenue']/new_df['trip_count']
    
    new_df['avg_dist_pmin'] = new_df['total_miles']/new_df['total_time_min']
    new_df['avg_rev_pmin'] = new_df['total_revenue']/new_df['total_time_min']
    
    return new_df


def clean_file(csv, taxi_zone_lkup_df, file_key, gb_field):
    
    chunks_df = pd.read_csv(csv, chunksize=1000000)    
    gb_field_list = ["PULocationID","total_amount","trip_time_min","trip_distance"]
    
    new_gb_df = pd.DataFrame()    
    
    for chunk_df in chunks_df:  
        clean_chunk_df = clean_chunk(chunk_df, taxi_zone_lkup_df, file_key)
        gb_clean_chunk_df = groupby(clean_chunk_df, gb_field, gb_field_list)
        # Once the data filtering is done, append the chunk to list
        new_gb_df = new_gb_df.append(gb_clean_chunk_df)
    return new_gb_df


def clean_chunk(df, zone_lkup_df, file_key):
    new_df = remove_unwanted_columns(df)
    new_df = remove_unwanted_rows(new_df, file_key)
    new_df = add_new_columns(new_df, zone_lkup_df)
    return new_df

def remove_unwanted_columns(df):
    new_df = df.drop(['VendorID', 'store_and_fwd_flag','fare_amount',
                    'extra', 'mta_tax','tolls_amount',
                    'improvement_surcharge'], axis=1) 
    return new_df

def remove_unwanted_rows(df, file_key):
    new_df = df 
    new_df['year'] = pd.DatetimeIndex(new_df['tpep_pickup_datetime']).year
    new_df['month'] = pd.DatetimeIndex(new_df['tpep_pickup_datetime']).month
    
    file_month = file_key.split("_")[1]
    file_year = file_key.split("_")[0] 

    new_df = new_df.loc[(new_df["month"] == float(file_month)) & 
                        (new_df["year"] == float(file_year)), :]

#     new_df = df.drop(['year', 'month'], axis=1)    

    return new_df

def add_new_columns(df, zone_lkup_df):
    #set convert lkup_df to dictionary to leverage 'map' method
    taxi_zone_dic = pd.Series(zone_lkup_df.Zone.values, index=zone_lkup_df.LocationID).to_dict()

    new_df = df
    #create 2 columns for zone_name
    new_df['PU_zone_name'] = new_df.PULocationID.map(taxi_zone_dic)
    new_df['DO_zone_name'] = new_df.DOLocationID.map(taxi_zone_dic)

    #create 2 columns for time
    new_df['trip_time'] = pd.to_datetime(new_df["tpep_dropoff_datetime"]) - pd.to_datetime(new_df["tpep_pickup_datetime"])
    new_df['trip_time_min'] = new_df['trip_time'].dt.total_seconds()/60
    
    new_df['day_of_week'] = pd.to_datetime(new_df["tpep_pickup_datetime"]).dt.day_name()
    new_df['hour_of_day'] = pd.to_datetime(new_df["tpep_pickup_datetime"]).dt.hour

    
    return new_df

def groupby(df, gb_field_name, fields):
    groupby = df.groupby([gb_field_name])
    gb_trip_rev_total = groupby[fields[1]].sum()
    gb_trip_time_total = groupby[fields[2]].sum()
    gb_trip_miles_total = groupby[fields[3]].sum()
    
    if fields[0] == 'trip_count':
        gb_trip_count = groupby[fields[0]].sum()
    else:
        gb_trip_count = groupby[fields[0]].count()
        
    new_df = pd.DataFrame({"trip_count": gb_trip_count,
                          "total_revenue": gb_trip_rev_total,
                          "total_time_min": gb_trip_time_total,
                          "total_miles": gb_trip_miles_total})
    
    return new_df

In [93]:
# Storing Filenames to Load
taxi_file_names = {
    "2018_01": os.path.join("..", "TaxiData", "yellow_tripdata_2018-01.csv"),
#     "2018_02": os.path.join("..", "TaxiData", "yellow_tripdata_2018-02.csv"),
#     "2018_03": os.path.join("..", "TaxiData", "yellow_tripdata_2018-03.csv"),
#     "2018_04": os.path.join("..", "TaxiData", "yellow_tripdata_2018-04.csv"),
#     "2018_05": os.path.join("..", "TaxiData", "yellow_tripdata_2018-05.csv"),
#     "2018_06": os.path.join("..", "TaxiData", "yellow_tripdata_2018-06.csv"),
#     "2018_07": os.path.join("..", "TaxiData", "yellow_tripdata_2018-07.csv"),
#     "2018_08": os.path.join("..", "TaxiData", "yellow_tripdata_2018-08.csv"),
#     "2018_09": os.path.join("..", "TaxiData", "yellow_tripdata_2018-09.csv"),
#     "2018_10": os.path.join("..", "TaxiData", "yellow_tripdata_2018-10.csv"),
#     "2018_11": os.path.join("..", "TaxiData", "yellow_tripdata_2018-11.csv"),
#     "2018_12": os.path.join("..", "TaxiData", "yellow_tripdata_2018-12.csv")
}

taxi_zone_lkup_csv = os.path.join("..","TaxiData", "taxi_zone_lookup.csv")

In [78]:
#################
'variable names'
#################
'PU_zone_name'
'month'    
'day_of_week'
'hour_of_day'    
'RatecodeID'     
#################

start = datetime.datetime.today()
# key_variables['PU_zone_name','month','day_of_week','hour_of_day','RatecodeID'] 
#retrieves a summary from a years worth of data for a given variable
df = retrieve_summary(taxi_zone_lkup_csv, taxi_file_names, "PU_zone_name")

end = datetime.datetime.today()

print(end - start)
df

#whatever

2018_01
2018_02
2018_03
2018_04
2018_05
2018_06
2018_07
2018_08
2018_09
2018_10
2018_11
2018_12
0:10:46.514662


trip_count  total_revenue  \
PU_zone_name                                                     
Allerton/Pelham Gardens                    1247   3.696660e+04   
Alphabet City                            212960   3.130816e+06   
Arden Heights                               131   1.044042e+04   
Arrochar/Fort Wadsworth                     405   1.317353e+04   
Astoria                                  165170   2.179969e+06   
Astoria Park                               1078   4.467773e+04   
Auburndale                                  891   2.268757e+04   
Baisley Park                              32530   1.989042e+06   
Bath Beach                                 1273   4.088744e+04   
Battery Park                              45985   9.480457e+05   
Battery Park City                        924223   1.828435e+07   
Bay Ridge                                  7566   2.333762e+05   
Bay Terrace/Fort Totten                     936   3.409094e+04   
Bayside                                    1641   4.965676e+04   
Bedford                                   20084   3.131562e+05   
Bedford Park                               2091   4.953410e+04   
Bellerose                                  1694   5.326238e+04   
Belmont                                    1086   2.806535e+04   
Bensonhurst East                           2186   6.905002e+04   
Bensonhurst West                           3077   9.973693e+04   
Bloomfield/Emerson Hill                     317   1.968920e+04   
Bloomingdale                             303766   4.241850e+06   
Boerum Hill                              106264   1.751516e+06   
Borough Park                               5355   1.283639e+05   
Breezy Point/Fort Tilden/Riis Beach          95   3.771390e+03   
Briarwood/Jamaica Hills                    8624   1.295918e+06   
Brighton Beach                             1663   5.226490e+04   
Broad Channel                                25   7.059000e+02   
Bronx Park                                  470   1.512723e+04   
Bronxdale                                  1830   5.810607e+04   
...                                         ...            ...   
Union Sq                                3207586   4.422206e+07   
University Heights/Morris Heights          4225   1.071299e+05   
Upper East Side North                   3861775   4.772714e+07   
Upper East Side South                   4257423   5.087882e+07   
Upper West Side North                   2001742   2.600361e+07   
Upper West Side South                   2633770   3.403415e+07   
Van Cortlandt Park                          432   1.473957e+04   
Van Cortlandt Village                      2017   5.210369e+04   
Van Nest/Morris Park                       2765   7.051559e+04   
Washington Heights North                  20460   4.209541e+05   
Washington Heights South                 105456   1.910024e+06   
West Brighton                                82   3.488890e+03   
West Chelsea/Hudson Yards               1433007   2.130707e+07   
West Concourse                            15042   2.754025e+05   
West Farms/Bronx River                     1538   3.915882e+04   
West Village                            2025102   2.808206e+07   
Westchester Village/Unionport              2107   5.227695e+04   
Westerleigh                                 132   5.327510e+03   
Whitestone                                 1147   5.040522e+04   
Willets Point                               322   1.561953e+04   
Williamsbridge/Olinville                   3326   1.072762e+05   
Williamsburg (North Side)                103435   1.701908e+06   
Williamsburg (South Side)                 78726   1.290001e+06   
Windsor Terrace                            4008   9.465190e+04   
Woodhaven                                  2260   7.104665e+04   
Woodlawn/Wakefield                         1964   6.856828e+04   
Woodside                                  56047   9.928367e+05   
World Trade Center                       599861   1.208178e+07   
Yorkville E

In [94]:
df_kev = retrieve_summary(taxi_zone_lkup_csv, taxi_file_names, "hour_of_day")
df_kev

2018_01


trip_count  total_revenue  total_time_min  total_miles  \
hour_of_day                                                           
0                259259   4.277610e+06    3.927610e+06    867288.41   
1                188641   2.986591e+06    2.863651e+06    600525.62   
2                142073   2.191263e+06    2.177018e+06    443468.76   
3                102587   1.641999e+06    1.528718e+06    343686.67   
4                 79349   1.442820e+06    1.266512e+06    321464.25   
5                 86213   1.654866e+06    1.306786e+06    371493.51   
6                196989   2.983422e+06    2.747263e+06    625414.08   
7                335644   4.792964e+06    5.134458e+06    896120.25   
8                418282   5.988820e+06    6.529072e+06   1005161.03   
9                420035   6.128046e+06    6.573012e+06   1017115.40   
10               405821   5.955326e+06    6.363754e+06   1010411.99   
11               418555   6.076489e+06    6.565715e+06   1205465.69   
12               447071   6.646253e+06    7.038735e+06   1134433.08   
13               449492   6.892589e+06    7.268146e+06   1210561.54   
14               479227   7.523222e+06    8.170983e+06   1332582.16   
15               490776   7.678843e+06    8.403647e+06   1346680.19   
16               463957   7.702528e+06    8.079068e+06   1302109.58   
17               517794   8.240654e+06    8.778382e+06   1353262.83   
18               575604   8.798321e+06    9.040730e+06   1431080.79   
19               543213   8.264167e+06    8.077683e+06   1409534.16   
20               481646   7.434479e+06    6.921806e+06   1375445.34   
21               475070   7.636418e+06    6.912084e+06   1456747.80   
22               439444   7.131376e+06    6.445970e+06   1383853.54   
23               342790   5.624548e+06    5.020154e+06   1117727.59   

             avg_min_ptrip  avg_dist_ptrip  avg_rev_ptrip  avg_dist_pmin  \
hour_of_day                                                                
0                15.149369        3.345259      16.499369       0.220818   
1                15.180428        3.183431      15.832143       0.209706   
2                15.323233        3.121415      15.423504       0.203705   
3                14.901677        3.350197      16.005919       0.224820   
4                15.961281        4.051270      18.183222       0.253819   
5                15.157650        4.309020      19.195085       0.284280   
6                13.946278        3.174868      15.145122       0.227650   
7                15.297332        2.669853      14.279904       0.174531   
8                15.609260        2.403070      14.317661       0.153952   
9                15.648723        2.421502      14.589369       0.154741   
10               15.681184        2.489797      14.674761       0.158776   
11               15.686623        2.880065      14.517779       0.183600   
12               15.744109        2.537479      14.866214       0.161170   
13               16.169690        2.693177      15.334176       0.166557   
14               17.050340        2.780691      15.698660       0.163087   
15               17.123183        2.743981      15.646329       0.160249   
16               17.413399        2.806531      16.601815       0.161171   
17               16.953425        2.613516      15.914928       0.154159   
18               15.706510        2.486225      15.285372       0.158293   
19               14.870194        2.594809      15.213493       0.174497   
20               14.371147        2.855718      15.435566       0.198712   
21               14.549611        3.066386      16.074300       0.210754   
22               14.668468        3.149101      16.228179       0.214685   
23               14.644983        3.260677      16.408144       0.222648   

             avg_rev_pmin  
hour_of_day                
0                1.089113  
1                1.042931  
2                1.006544  
3                1.074102  
4                1.13

In [92]:
df_kev = retrieve_summary(taxi_zone_lkup_csv, taxi_file_names, "day_of_week")
df_kev

2018_12


trip_count  total_revenue  total_time_min  total_miles  \
day_of_week                                                           
Friday          1146734   1.939565e+07    2.253313e+07   3320120.65   
Monday          1212860   1.949965e+07    2.093556e+07   3505801.78   
Saturday        1388930   2.172764e+07    2.603594e+07   3838846.86   
Sunday          1194161   1.913649e+07    2.089952e+07   3633013.02   
Thursday        1134030   1.978058e+07    2.306866e+07   3319028.20   
Tuesday         1005519   1.674726e+07    1.881055e+07   2877540.07   
Wednesday       1090078   1.875336e+07    2.087849e+07   3144290.17   

             avg_min_ptrip  avg_dist_ptrip  avg_rev_ptrip  avg_dist_pmin  \
day_of_week                                                                
Friday           19.649831        2.895284      16.913815       0.147344   
Monday           17.261317        2.890525      16.077414       0.167457   
Saturday         18.745321        2.763888      15.643439       0.147444   
Sunday           17.501429        3.042314      16.025048       0.173832   
Thursday         20.342191        2.926755      17.442731       0.143876   
Tuesday          18.707308        2.861746      16.655342       0.152975   
Wednesday        19.153206        2.884463      17.203687       0.150600   

             avg_rev_pmin  
day_of_week                
Friday           0.860761  
Monday           0.931413  
Saturday         0.834525  
Sunday           0.915642  
Thursday         0.857466  
Tuesday          0.890312  
Wednesday        0.898214

In [88]:
df2_high_volume= df.loc[df['trip_count'] > 100000, :]


df2_high_volume.sort_values(by=['avg_rev_pmin'],ascending= False)

trip_count  total_revenue  total_time_min  \
PU_zone_name                                                                
Long Island City/Hunters Point      123842   2.268457e+06    1.592470e+06   
JFK Airport                        2576277   1.461847e+08    1.065117e+08   
LaGuardia Airport                  2883849   1.229799e+08    1.027026e+08   
NV                                 1613325   2.850649e+07    2.539279e+07   
Sunnyside                           162214   2.975396e+06    2.805894e+06   
Washington Heights South            105456   1.910024e+06    1.869038e+06   
Yorkville East                     1202355   1.601837e+07    1.624714e+07   
Long Island City/Queens Plaza       102298   1.480831e+06    1.502721e+06   
Yorkville West                     1907155   2.450514e+07    2.488460e+07   
Downtown Brooklyn/MetroTech         135785   2.504613e+06    2.562204e+06   
Park Slope                          100338   1.642225e+06    1.692981e+06   
Battery Park City                   924223   1.828435e+07    1.885111e+07   
Financial District North            775807   1.620382e+07    1.680304e+07   
Chinatown                           229822   4.675450e+06    4.862972e+06   
Astoria                             165170   2.179969e+06    2.271282e+06   
Brooklyn Heights                    111753   2.086462e+06    2.175505e+06   
Stuy Town/Peter Cooper Village      229120   3.232091e+06    3.376055e+06   
East Harlem South                   729348   9.488882e+06    9.960323e+06   
Upper West Side North              2001742   2.600361e+07    2.739845e+07   
Boerum Hill                         106264   1.751516e+06    1.845770e+06   
Manhattan Valley                    898049   1.192208e+07    1.258219e+07   
Bloomingdale                        303766   4.241850e+06    4.492163e+06   
Lenox Hill West                    2414269   2.956315e+07    3.138815e+07   
Lenox Hill East                    1771643   2.467884e+07    2.622168e+07   
Financial District South            406697   8.939178e+06    9.506524e+06   
Morningside Heights                 536741   7.976989e+06    8.503619e+06   
Manhattanville                      101239   1.428281e+06    1.524741e+06   
East Harlem North                   408278   5.283401e+06    5.658004e+06   
UN/Turtle Bay South                1349670   2.074010e+07    2.228149e+07   
Central Harlem                      397465   5.194667e+06    5.581433e+06   
...                                    ...            ...             ...   
Murray Hill                        3311295   4.842938e+07    5.311875e+07   
Hudson Sq                           593322   9.056551e+06    9.956116e+06   
Williamsburg (North Side)           103435   1.701908e+06    1.874223e+06   
Midtown East                       3608878   5.352629e+07    5.910578e+07   
Gramercy                           2442518   3.361400e+07    3.720815e+07   
Alphabet City                       212960   3.130816e+06    3.486901e+06   
TriBeCa/Civic Center               1762756   2.886836e+07    3.221677e+07   
Flatiron                           1845193   2.510150e+07    2.804653e+07   
Upper East Side South              4257423   5.087882e+07    5.685669e+07   
Clinton East                       3266079   4.671411e+07    5.233910e+07   
West Chelsea/Hudson Yards          1433007   2.130707e+07    2.408560e+07   
Lincoln Square East                3077284   4.147997e+07    4.714004e+07   
Midtown South                      2466006   3.628769e+07    4.141745e+07   
West Village                       2025102   2.808206e+07    3.229339e+07   
World Trade Center                  599861   1.208178e+07    1.389989e+07   
Times Sq/Theatre District          3485281   5.711996e+07    6.578085e+07   
Garment District                   2135527   3.135223e+07    3.613774e+07   
Lower East Side                    1369477   2.145399e+07    2.491063e+07   
East Chelsea                       2398039   3.484665e+07    4.054861e+07   
Midtown North               

In [79]:
trip_count = df['trip_count'].sum()
print("{:,}".format(trip_count))

102,747,765


In [69]:
############ MESS AROUND BELOW HERE ##############################

gb = df.reset_index()
gb

PU_zone_name  trip_count  total_revenue  \
0                Allerton/Pelham Gardens          37   7.076100e+02   
1                          Alphabet City       19656   2.740243e+05   
2                          Arden Heights           2   2.217600e+02   
3                Arrochar/Fort Wadsworth          51   1.508050e+03   
4                                Astoria       14673   1.888605e+05   
5                           Astoria Park          71   2.986610e+03   
6                             Auburndale          46   9.376000e+02   
7                           Baisley Park        2079   1.299668e+05   
8                             Bath Beach          63   1.493400e+03   
9                           Battery Park        2934   5.642171e+04   
10                     Battery Park City       75611   1.423004e+06   
11                             Bay Ridge         382   9.168990e+03   
12               Bay Terrace/Fort Totten          30   6.014600e+02   
13                               Bayside          57   1.420770e+03   
14                               Bedford        1701   2.248057e+04   
15                          Bedford Park         105   1.942610e+03   
16                             Bellerose          94   1.921280e+03   
17                               Belmont          50   1.170450e+03   
18                      Bensonhurst East          68   1.447300e+03   
19                      Bensonhurst West         119   2.659730e+03   
20               Bloomfield/Emerson Hill           7   3.187600e+02   
21                          Bloomingdale       25353   3.401959e+05   
22                           Boerum Hill        8514   1.333933e+05   
23                          Borough Park         288   4.726250e+03   
24   Breezy Point/Fort Tilden/Riis Beach           5   1.572800e+02   
25               Briarwood/Jamaica Hills         544   2.430803e+04   
26                        Brighton Beach          90   2.747930e+03   
27                         Broad Channel           4   7.181000e+01   
28                            Bronx Park          21   7.723500e+02   
29                             Bronxdale          45   8.368600e+02   
..                                   ...         ...            ...   
227                             Union Sq      283960   3.745361e+06   
228    University Heights/Morris Heights         273   4.599050e+03   
229                Upper East Side North      345684   4.130205e+06   
230                Upper East Side South      360976   4.134051e+06   
231                Upper West Side North      176926   2.202865e+06   
232                Upper West Side South      228251   2.803691e+06   
233                   Van Cortlandt Park          18   2.968000e+02   
234                Van Cortlandt Village          85   1.534270e+03   
235                 Van Nest/Morris Park         119   2.065090e+03   
236             Washington Heights North        1560   2.939892e+04   
237             Washington Heights South        8560   1.472368e+05   
238                        West Brighton           5   3.942800e+02   
239            West Chelsea/Hudson Yards      118513   1.680254e+06   
240                       West Concourse        1171   1.830661e+04   
241               West Farms/Bronx River         104   1.888060e+03   
242                         West Village      175800   2.340402e+06   
243        Westchester Village/Unionport          95   1.825730e+03   
244                          Westerleigh           7   1.488600e+02   
245                           Whitestone          67   2.058300e+03   
246                        Willets Point          23   2.105160e+03   
247             Williamsbridge/Olinville         105   2.321800e+03   
248            Williamsburg (North Side)        7795   1.199856e+05   
249            Williamsburg (South Side)        6545   1.041417e+05   
250                      Windsor Terrace         256   5.502920e+03   
251                            Woodhaven          98

In [70]:
gb['PU_zone_name'].unique

<bound method Series.unique of 0                  Allerton/Pelham Gardens
1                            Alphabet City
2                            Arden Heights
3                  Arrochar/Fort Wadsworth
4                                  Astoria
5                             Astoria Park
6                               Auburndale
7                             Baisley Park
8                               Bath Beach
9                             Battery Park
10                       Battery Park City
11                               Bay Ridge
12                 Bay Terrace/Fort Totten
13                                 Bayside
14                                 Bedford
15                            Bedford Park
16                               Bellerose
17                                 Belmont
18                        Bensonhurst East
19                        Bensonhurst West
20                 Bloomfield/Emerson Hill
21                            Bloomingdale
22                     

In [72]:
UES_df = df.loc[['Upper East Side North','Upper East Side South','Yorkville East']]

UES_df

trip_count  total_revenue  total_time_min  total_miles  \
PU_zone_name                                                                    
Upper East Side North      345684   4.130205e+06      4410053.10    624045.73   
Upper East Side South      360976   4.134051e+06      4396395.00    608925.50   
Yorkville East             109302   1.398951e+06      1371767.25    237985.19   

                       avg_time_min_per_trip  avg_distance_per_trip  \
PU_zone_name                                                          
Upper East Side North              12.757470               1.805249   
Upper East Side South              12.179189               1.686886   
Yorkville East                     12.550248               2.177318   

                       avg_rev_per_trip  avg_distance_per_min  avg_rev_per_min  
PU_zone_name                                                                    
Upper East Side North         11.947922              0.141505         0.936543  
Upper East Side South         11.452426              0.138506         0.940327  
Yorkville East                12.798954              0.173488         1.019817

In [34]:
df = pd.read_csv(taxi_file_names["2018_12"])  
df =remove_unwanted_columns(df)
df = remove_unwanted_rows(df,"2018_12")
df = add_new_columns(df,taxi_zone_lkup_df)
df.head()

tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  \
0  2018-12-01 00:28:22   2018-12-01 00:44:07                2            2.5   
1  2018-12-01 00:52:29   2018-12-01 01:11:37                3            2.3   
2  2018-12-01 00:12:52   2018-12-01 00:36:23                1            0.0   
3  2018-12-01 00:35:08   2018-12-01 00:43:11                1            3.9   
4  2018-12-01 00:21:54   2018-12-01 01:15:13                1           12.8   

   RatecodeID  PULocationID  DOLocationID  payment_type  tip_amount  \
0           1           148           234             1        3.95   
1           1           170           144             1        2.85   
2           1           113           193             2        0.00   
3           1            95            92             1        2.75   
4           1           163           228             1        9.25   

   total_amount  year  month             PU_zone_name  \
0         17.25  2018     12          Lower East Side   
1         17.15  2018     12              Murray Hill   
2          3.80  2018     12  Greenwich Village North   
3         16.55  2018     12             Forest Hills   
4         55.55  2018     12            Midtown North   

              DO_zone_name trip_time  trip_time_min day_of_week  hour_of_day  
0                 Union Sq  00:15:45      15.750000    Saturday            0  
1      Little Italy/NoLiTa  00:19:08      19.133333    Saturday            0  
2  Queensbridge/Ravenswood  00:23:31      23.516667    Saturday            0  
3                 Flushing  00:08:03       8.050000    Saturday            0  
4         Sunset Park West  00:53:19      53.316667    Saturday            0

array([ 0,  1, 23, 21,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
       15, 16, 17, 18, 19, 20, 22], dtype=int64)

In [6]:

jfk_df = gb_final_file_df.loc[['JFK Airport','Newark Airport','LaGuardia Airport']]

jfk_df

trip_count   sum_revenue  total_time_min  total_miles
PU_zone_name                                                            
JFK Airport            190098  1.075741e+07    8.828474e+06   2978933.35
Newark Airport            664  6.201346e+04    3.830567e+03      1313.39
LaGuardia Airport      209668  8.870254e+06    7.839671e+06   2004762.35

In [ ]:
# print(time)
jfk_df = gb_final_file_df.loc[['JFK Airport','Newark Airport','LaGuardia Airport']]

jfk_df

In [ ]:
gb_final_file_df.info(memory_usage='deep')

In [ ]:
df_concat.info(memory_usage='deep')

In [ ]:

# groupby_PUzone_taxi_201801 = taxi_201801_df.groupby(["PU_zone_name"])
# gb_PU_trip_count = groupby_PUzone_taxi_201801['PULocationID'].count()
# gb_PU_trip_rev_total = groupby_PUzone_taxi_201801['total_amount'].sum()
# gb_PU_trip_time_total = groupby_PUzone_taxi_201801['trip_time_min'].sum()
# gb_PU_trip_miles_total = groupby_PUzone_taxi_201801['trip_distance'].sum()

gb_PU_trip_time_avg = groupby_PUzone_taxi_201801['trip_time_min'].mean()
gb_PU_trip_rev_avg = groupby_PUzone_taxi_201801['total_amount'].mean()
gb_PU_trip_miles_avg = groupby_PUzone_taxi_201801['trip_distance'].mean()

gb_PU_summary_df = pd.DataFrame({"trip_count": gb_PU_trip_count,
                             "avg_revenue": gb_PU_trip_rev_avg,
                             "sum_revenue": gb_PU_trip_rev_total,
                             "total_time_min": gb_PU_trip_time_total,
                             "avg_time_min": gb_PU_trip_time_avg,
                             "total_miles": gb_PU_trip_miles_total,
                             "avg_miles": gb_PU_trip_miles_avg})

gb_PU_summary_df['avg_rev_per_min'] = gb_PU_summary_df['sum_revenue']/gb_PU_summary_df['total_time_min'] 
gb_PU_summary_df['avg_rev_per_min']


gb_PU_summary_df = gb_PU_summary_df.sort_values('avg_rev_per_min', ascending=False)

jfk_df = gb_PU_summary_df.loc[['JFK Airport','Newark Airport','LaGuardia Airport']]

jfk_df

# gb_PU_summary_df

In [ ]:
gb_PU_summary_df.info(memory_usage='deep')

In [91]:
df_3 = retrieve_summary(taxi_zone_lkup_csv, taxi_file_names, "PU_zone_name")
df_3

2018_12


trip_count  total_revenue  \
PU_zone_name                                                     
Allerton/Pelham Gardens                     163   5.330920e+03   
Alphabet City                             16666   2.461212e+05   
Arden Heights                                14   1.093760e+03   
Arrochar/Fort Wadsworth                      37   1.472160e+03   
Astoria                                   13141   1.738028e+05   
Astoria Park                                 86   3.501480e+03   
Auburndale                                  142   3.849810e+03   
Baisley Park                               2742   1.560588e+05   
Bath Beach                                  128   4.155250e+03   
Battery Park                               3145   6.427123e+04   
Battery Park City                         67590   1.347362e+06   
Bay Ridge                                   867   3.098099e+04   
Bay Terrace/Fort Totten                      98   3.567930e+03   
Bayside                                     225   7.522120e+03   
Bedford                                    1654   2.874079e+04   
Bedford Park                                267   7.006430e+03   
Bellerose                                   144   5.790570e+03   
Belmont                                     143   3.814440e+03   
Bensonhurst East                            264   7.805400e+03   
Bensonhurst West                            363   1.118381e+04   
Bloomfield/Emerson Hill                      62   3.640940e+03   
Bloomingdale                              24423   3.373928e+05   
Boerum Hill                                8335   1.399430e+05   
Borough Park                                689   1.660407e+04   
Breezy Point/Fort Tilden/Riis Beach           3   4.562000e+01   
Briarwood/Jamaica Hills                     871   3.865748e+04   
Brighton Beach                              209   6.439060e+03   
Broad Channel                                 2   1.043600e+02   
Bronx Park                                   42   1.430340e+03   
Bronxdale                                   275   8.946670e+03   
...                                         ...            ...   
Union Sq                                 238101   3.335165e+06   
University Heights/Morris Heights           574   1.559640e+04   
Upper East Side North                    333049   4.090339e+06   
Upper East Side South                    360722   4.374277e+06   
Upper West Side North                    167771   2.156394e+06   
Upper West Side South                    225357   2.903295e+06   
Van Cortlandt Park                           50   1.578950e+03   
Van Cortlandt Village                       274   8.198790e+03   
Van Nest/Morris Park                        379   1.070745e+04   
Washington Heights North                   1548   3.465125e+04   
Washington Heights South                   8534   1.569691e+05   
West Brighton                                11   4.451600e+02   
West Chelsea/Hudson Yards                 98136   1.487238e+06   
West Concourse                             1100   1.992216e+04   
West Farms/Bronx River                      205   5.726610e+03   
West Village                             162375   2.295988e+06   
Westchester Village/Unionport               302   7.714790e+03   
Westerleigh                                  18   5.810300e+02   
Whitestone                                  146   5.426590e+03   
Willets Point                                21   1.103020e+03   
Williamsbridge/Olinville                    481   1.585882e+04   
Williamsburg (North Side)                  7030   1.154336e+05   
Williamsburg (South Side)                  5100   8.619844e+04   
Windsor Terrace                             420   1.021930e+04   
Woodhaven                                   305   1.016696e+04   
Woodlawn/Wakefield                          256   9.874130e+03   
Woodside                                   4497   8.266999e+04   
World Trade Center                        49885   1.015961e+06   
Yorkville E

In [53]:
# datetime.date(d.year, d.month+1, d.day) - datetime.timedelta(1)
import datetime

# d = datetime.datetime.today()
# int(str(datetime.date(d.year, d.month+1, 1) - datetime.timedelta(1)))

print(datetime.datetime.today())

2019-06-09 17:04:25.667129
